<a href="https://colab.research.google.com/github/bvprasan/GLDN/blob/main/POS%20Tagging%20using%20LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
text = nltk.word_tokenize('I came to Bangaluru to meet some awesome people.')

In [7]:
nltk.pos_tag(text)

[('I', 'PRP'),
 ('came', 'VBD'),
 ('to', 'TO'),
 ('Bangaluru', 'NNP'),
 ('to', 'TO'),
 ('meet', 'VB'),
 ('some', 'DT'),
 ('awesome', 'JJ'),
 ('people', 'NNS'),
 ('.', '.')]

In [8]:
text = nltk.word_tokenize('I am walking.')

In [9]:
nltk.pos_tag(text)

[('I', 'PRP'), ('am', 'VBP'), ('walking', 'VBG'), ('.', '.')]

In [10]:
text = nltk.word_tokenize('Walking is good for health.')

In [11]:
nltk.pos_tag(text)

[('Walking', 'NN'),
 ('is', 'VBZ'),
 ('good', 'JJ'),
 ('for', 'IN'),
 ('health', 'NN'),
 ('.', '.')]

In [12]:
text = nltk.Text(word.lower() for word in nltk.corpus.brown.words())

In [13]:
text.similar('kid')

man other time house trial way place doctor world car city state one
past children case family government south situation


In [14]:
text.similar('run')

get be do in see work go have take make put and find time look day say
use come show


In [15]:
text.similar('on')

in of to and for at with from by as that is into but when was over
about all through


In [16]:
text.similar('cricket')

and available second wonderful glass moment window formidable wicked


In [17]:
nltk.pos_tag(nltk.word_tokenize('cricket'))

[('cricket', 'NN')]

In [18]:
%tensorflow_version 2.x

In [19]:
import tensorflow
tensorflow.__version__

'2.5.0'

In [20]:
import random

In [21]:
random.seed(0)

In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
from google.colab import drive

In [24]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [25]:
import pandas as pd

In [26]:
import numpy as np

In [32]:
data = pd.read_csv('/content/drive/My Drive/ner_dataset (1).csv', encoding='latin1')

In [33]:
data = data.fillna(method="ffill")

In [34]:
tags = list(set(data["POS"].values))

In [35]:
tags

['WDT',
 ':',
 'WRB',
 'MD',
 'POS',
 'VBZ',
 'NNPS',
 'WP$',
 ';',
 'VBP',
 'CC',
 'JJR',
 'RBR',
 'VBG',
 'VBD',
 'DT',
 ',',
 'PRP$',
 '``',
 '.',
 'FW',
 'VB',
 'VBN',
 'JJ',
 'EX',
 '$',
 'PRP',
 'NNS',
 'IN',
 'NNP',
 'NN',
 'RP',
 'WP',
 'PDT',
 'CD',
 'JJS',
 'UH',
 'RB',
 'RBS',
 'RRB',
 'LRB',
 'TO']

In [36]:
words = list(set(data["Word"].values))

In [37]:
words.append("DUMMY")

In [38]:
class ReadSentences(object):
  def __init__(self, data):
    self.data = data
    self.empty = False
    agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                       s["POS"].values.tolist(),
                                                       s["Tag"].values.tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]

In [39]:
sentences = ReadSentences(data).sentences

In [40]:
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
max_len = 50
X = [[word2id[w[0]] for w in s] for s in sentences]

In [43]:
len(X)

47959

In [44]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=len(words)-1)

In [45]:
len(X)

47959

In [46]:
y = [[tag2id[w[1]] for w in s] for s in sentences]

In [47]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2id["."])

In [48]:
from tensorflow.keras.utils import to_categorical

In [49]:
y = [to_categorical(i, num_classes=len(tags)) for i in y]

In [50]:
y[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20)

In [53]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input

In [54]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=len(words), output_dim=50, input_length=max_len)(input)

In [55]:
model = Dropout(0.1)(model)

In [56]:
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)

In [57]:
out = TimeDistributed(Dense(len(tags), activation="softmax"))(model)

In [58]:
model = Model(input, out)

In [59]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [60]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=3, validation_split=0.1, verbose=1)

Epoch 1/3
1080/1080 [==============================] - 205s 185ms/step - loss: 0.3787 - accuracy: 0.8961 - val_loss: 0.0635 - val_accuracy: 0.9810
Epoch 2/3
1080/1080 [==============================] - 200s 185ms/step - loss: 0.0490 - accuracy: 0.9856 - val_loss: 0.0422 - val_accuracy: 0.9871
Epoch 3/3
1080/1080 [==============================] - 200s 185ms/step - loss: 0.0337 - accuracy: 0.9899 - val_loss: 0.0365 - val_accuracy: 0.9887


In [61]:
len(X_te)

9592

In [62]:
i = 1213
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
for w, pred in zip(X_te[i], p[0]):
  print("{:20} -- {}".format(words[w], tags[pred]))

U.S                  -- NNP
Open                 -- NNP
women                -- NNS
's                   -- POS
tennis               -- NN
champion             -- NN
Kim                  -- NNP
Clijsters            -- NNP
of                   -- IN
Belgium              -- NNP
has                  -- VBZ
defeated             -- VBN
French               -- JJ
player               -- NN
Nathalie             -- NNP
Dechy                -- NNP
to                   -- TO
reach                -- VB
the                  -- DT
finals               -- NNS
of                   -- IN
the                  -- DT
FORTIS               -- NNP
Championships        -- NNP
in                   -- IN
Luxembourg           -- NNP
.                    -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                --

In [63]:
sentence = nltk.word_tokenize('That was a nice jump')
X_Samp = pad_sequences(maxlen=max_len, sequences=[[word2id[word] for word in sentence]], padding="post", value=word2id["."])

In [64]:
X_Samp

array([[13871, 28604,  6207, 15141,  9093, 27976, 27976, 27976, 27976,
        27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976,
        27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976,
        27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976,
        27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976, 27976,
        27976, 27976, 27976, 27976, 27976]], dtype=int32)

In [65]:
p = model.predict(np.array([X_Samp[0]]))
p = np.argmax(p, axis=-1)
for w, pred in zip(X_Samp[0], p[0]):
  print("{:20} -- {}".format(words[w], tags[pred]))

That                 -- DT
was                  -- VBD
a                    -- DT
nice                 -- JJ
jump                 -- NN
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.                    -- .
.     